In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
# Library imports.
from pyswmm import Simulation, Links, Output
from swmm.toolkit.shared_enum import LinkAttribute, NodeAttribute, SubcatchAttribute
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import numpy.random as nprnd
from scipy.interpolate import griddata
from sklearn.linear_model import LinearRegression
import time

# Local imports.
from urbansurge import swmm_model, analysis_tools, file_utils

In [11]:
fault_type = 'roughness'

# Diameter fault.
fault_value_bulk = {24: [0, 0.2, 0.4, 0.6, 0.8]}

# Roughness fault.
fault_value_bulk = {23: [0.01, 0.02, 0.03, 0.04, 0.05]}

In [12]:
# Path to configuration file.
config_path = r"C:\Users\ay434\Box\Research\Smart_Stormwater\SWMM\Single_Pipe\single_pipe_config.yml"

for fault_component, fault_values in fault_value_bulk.items():

    output_df_list = []
    for fault_value in fault_values:
        print(f'FAULT VALUE: {fault_value}')

        # Create instance of SWMM model.
        swmm = swmm_model.SWMM(config_path)

        # Configure model.
        swmm.configure_model()

        if fault_type == 'diameter':
            # Assign fault variables.
            weir_id = fault_component
            offset = fault_value

            # Get initial weir height.
            initial_geometry = swmm.get_weir_geometry(weir_id)
            initial_height = initial_geometry[1]
            
            # Get initial weir crest height.
            initial_crest_height = swmm.get_weir_property(weir_id, 'CrestHt')

            # Update the weir height.
            update_geometry = initial_geometry.copy()
            update_geometry[1] = initial_height - offset
            swmm.set_weir_geometry(weir_id, update_geometry)
            
            # Update the weir crest height.
            swmm.set_weir_property(weir_id, 'CrestHt', offset)

        elif fault_type == 'roughness':
            # Assign fault variables.
            link_id = fault_component
            n = fault_value

            # Update the link roughness.
            swmm.set_link_roughness(link_id, n)

        elif fault_type == 'seepage':
            # Assign fault variables.
            link_id = fault_component
            seepage_rate = fault_value

            # Update the link roughness.
            swmm.set_link_seepage(link_id, seepage_rate)

        elif fault_type == 'infiltration':
            # Assign fault variables.
            storage_id = fault_component
            Ksat = fault_value

            # Update the saturated hydraulic conductivity.
            swmm.set_storage_property(storage_id, 'Ksat', Ksat)

        elif fault_type == 'silting':
            # Assign fault variables.
            storage_id = fault_component
            silt_depth = fault_value

            # Get storage depth.
            Ds_init = swmm.get_storage_property(storage_id, 'MaxDepth')

            # Get conduit offsets.
            overflow_link_id = swmm.get_storage_outfall_link(storage_id)
            (in_offset_init, out_offset) = swmm.get_link_offsets(overflow_link_id)

            # Get the storage invert elevation.
            zi_init = swmm.get_storage_property(storage_id, 'Elev.')

            # Update the storage_depth.
            Ds = Ds_init - silt_depth
            swmm.set_storage_property(storage_id, 'MaxDepth', Ds)

            # Update conduit offset.
            in_offset = in_offset_init - silt_depth
            swmm.set_link_offsets(overflow_link_id, (in_offset, out_offset))

            # Update storage invert elevation.
            zi = zi_init + silt_depth
            swmm.set_storage_property(storage_id, 'Elev.', zi)

        # Run the simulation.
        swmm.run_simulation()

        # OUTPUT TO DATA FRAME
        # ---------------------------------------------------------------------
        # Get the node depths.
        node_depth_df = swmm.get_node_depth()

        # Get node flooding.
        node_flood_df = swmm.get_node_flooding()

        # Get node inflow.
        node_total_inflow_df = swmm.get_node_total_inflow()

        # Get link flow.
        link_flow_df = swmm.get_link_flow()

        # Get link depth.
        link_depth_df = swmm.get_link_depth()

        # Get link velocity.
        link_velocity_df = swmm.get_link_velocity()

        # Get precipitation time series.
        prcp_df = swmm.get_rainfall_timeseries()

        # Join data frames.
        fault_df = node_depth_df.merge(node_flood_df, on='datetime')
        fault_df = fault_df.merge(node_total_inflow_df, on='datetime')
        fault_df = fault_df.merge(link_flow_df, on='datetime')
        fault_df = fault_df.merge(link_depth_df, on='datetime')
        fault_df = fault_df.merge(link_velocity_df, on='datetime')
        fault_df = fault_df.merge(prcp_df, on='datetime')

        # Add fault type, component, and value.
        fault_df['fault_type'] = fault_type
        fault_df['fault_component'] = fault_component
        fault_df['fault_value'] = fault_value
        fault_df = fault_df.copy()

        output_df_list.append(fault_df)

    fault_df = pd.concat(output_df_list)

    save_fp = r"C:\Users\ay434\Box\Research\Smart_Stormwater\SWMM\Single_Pipe\Output\{}_{}.csv".format(fault_type, fault_component)
    # save_fp = r"C:\Users\ay434\Box\Research\NSF_SCC\SWMM\Canandaigua\Output\healthy_physical_system_p1.csv"
    print(fault_df.shape)
    print(save_fp)
    fault_df.to_csv(save_fp, index=False)

FAULT VALUE: 0.01
Simulation info
Flow Units: CFS
System Units: US
Start Time: 2024-04-01 00:00:00
Start Time: 2024-04-01 12:00:00
Updated Roughness value to 0.01 for 23 in 23
Set Link 23 roughness to 0.01
INP PATH: C:\Users\ay434\Box\Research\Smart_Stormwater\SWMM\Single_Pipe\single_pipe_tmp.inp
2024-04-01 00:00:01 , 0
2024-04-01 00:16:41 , 2
2024-04-01 00:33:21 , 5
2024-04-01 00:50:01 , 7
2024-04-01 01:06:41 , 9
2024-04-01 01:23:21 , 12
2024-04-01 01:40:01 , 14
2024-04-01 01:56:41 , 16
2024-04-01 02:13:21 , 19
2024-04-01 02:30:01 , 21
2024-04-01 02:46:41 , 23
2024-04-01 03:03:21 , 25
2024-04-01 03:20:01 , 28
2024-04-01 03:36:41 , 30
2024-04-01 03:53:21 , 32
2024-04-01 04:10:01 , 35
2024-04-01 04:26:41 , 37
2024-04-01 04:43:21 , 39
2024-04-01 05:00:01 , 42
2024-04-01 05:16:41 , 44
2024-04-01 05:33:21 , 46
2024-04-01 05:50:01 , 49
2024-04-01 06:06:41 , 51
2024-04-01 06:23:21 , 53
2024-04-01 06:40:01 , 56
2024-04-01 06:56:41 , 58
2024-04-01 07:13:21 , 60
2024-04-01 07:30:01 , 63
2024-04